### Modules importeren

In [19]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

import os
from loguru import logger # pip install loguru
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

servername = r'DESKTOP-IT4OHPV\SQLEXPRESS' # PAS DIT AAN NAAR JE EIGEN SERVER!!!


# Data inlezen

### Connecties maken

In [21]:
# Functie om een connectie te maken met een SQL Server database
def establish_connection(servername, database):
    logger.info(f'Establishing connection with {database} on {servername}...')
    try:
        conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + servername + 
                        ';DATABASE=' + database + ';Trusted_Connection=yes')
    except Exception as e:
        logger.error(f'Error: {e}')
        logger.error(f'Failed to establish connection with {database} on {servername}.')
        return None
    
    return conn

# Functie om een connectie te maken met een Microsoft Access database
def establish_access_connection(database):
    db_name = os.path.basename(database)
    logger.info(f'Establishing connection with {db_name}...')
    try:
        conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + database)
    except Exception as e:
        logger.error(f'Error: {e}')
        logger.error(f'Failed to establish connection with {db_name}.')
        return None
    
    return conn

# Connecties maken met de bronnen
northwind_ssms_conn = establish_connection(servername, 'northwind')
if northwind_ssms_conn is not None:
    northwind_ssms_cursor = northwind_ssms_conn.cursor()
else:
    logger.error('Failed to create cursor for Northwind database.')

adventureworks_ssms_conn = establish_connection(servername, 'AdventureWorks2019')
if adventureworks_ssms_conn is not None:
    adventureworks_ssms_cursor = adventureworks_ssms_conn.cursor()
else:
    logger.error('Failed to create cursor for AdventureWorks2019 database.')

aenc_access_conn = establish_access_connection('../data/raw/aenc.accdb')
if aenc_access_conn is not None:
    aenc_access_cursor = aenc_access_conn.cursor()
else:
    logger.error('Failed to create cursor for aenc database.')

# SQLite, maak nieuwe databases aan als deze nog niet bestaat
logger.info('Establishing connection with SQLite databases...')
northwind_sqlite_conn = sqlite3.connect('../data/processed/northwind.sqlite')
adventureworks_sqlite_conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
aenc_sqlite_conn = sqlite3.connect('../data/processed/aenc.sqlite')
logger.success('Connections established!')

2024-05-06 17:29:08.656 | INFO     | __main__:establish_connection:3 - Establishing connection with northwind on DESKTOP-IT4OHPV\SQLEXPRESS...
2024-05-06 17:29:08.667 | INFO     | __main__:establish_connection:3 - Establishing connection with AdventureWorks2019 on DESKTOP-IT4OHPV\SQLEXPRESS...
2024-05-06 17:29:08.679 | INFO     | __main__:establish_access_connection:17 - Establishing connection with aenc.accdb...
2024-05-06 17:29:08.680 | ERROR    | __main__:establish_access_connection:21 - Error: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
2024-05-06 17:29:08.680 | ERROR    | __main__:establish_access_connection:22 - Failed to establish connection with aenc.accdb.
2024-05-06 17:29:08.681 | ERROR    | __main__:<module>:44 - Failed to create cursor for aenc database.
2024-05-06 17:29:08.682 | INFO     | __main__:<module>:47 - Establishing connection with SQLite databases...
2024-05-06 17:29:08.68

### Bron-databases overzetten naar SQLite bestanden

In [15]:
# Functie om tabellen van een bron-database over te zetten naar een SQLite database
def transfer_tables_to_sqlite(tables, conn, sqlite_conn):
    db_name = sqlite_conn.execute("PRAGMA database_list;").fetchall()[0][2] # Padlocatie van de database
    db_name = os.path.basename(db_name) # Alleen de naam van de database
    logger.info(f'Transferring tables to SQLite database {db_name}...')

    failed_tables = []

    for table in tables:
        try:
            sql_query = f'SELECT * FROM {table};'
            data = pd.read_sql(sql_query, conn)
            sqlite_table_name = table.replace('[', '').replace(']', '')  # Verwijder vierkante haken voor tabellen met een spatie
            data.to_sql(sqlite_table_name, sqlite_conn, index=False, if_exists='replace')
        except Exception as e:
            logger.error(f'Error transferring table {table} to SQLite database {db_name}. Error: {e}')
            failed_tables.append(table)
            break
    
    if len(failed_tables) > 0:
        logger.warning(f'Tables that failed to transfer: {failed_tables}. Rest of the tables were transferred successfully to SQLite database {db_name}.')
    else:
        logger.success(f'Transferred tables to SQLite database {db_name}!')


# Tabellen die worden overgezet naar SQLite
northwind_tables = [
    'Categories', 
    'CustomerCustomerDemo', 
    'CustomerDemographics', 
    'Customers', 
    'Employees', 
    'EmployeeTerritories', 
    '[Order Details]', # Vanwege de spatie in de tabelnaam moet deze tussen vierkante haken (I <3 PYTHON!!!)
    'Orders', 
    'Products', 
    'Region', 
    'Shippers', 
    'Suppliers', 
    'Territories'
    ]

adventureworks_tables = [
    'dbo.AWBuildVersion',
    'dbo.DatabaseLog',
    'dbo.ErrorLog',
    'HumanResources.Department',
    'HumanResources.Employee',
    'HumanResources.EmployeeDepartmentHistory',
    'HumanResources.EmployeePayHistory',
    'HumanResources.JobCandidate',
    'HumanResources.Shift',
    'Person.Address',
    'Person.AddressType',
    'Person.BusinessEntity',
    'Person.BusinessEntityAddress',
    'Person.BusinessEntityContact',
    'Person.ContactType',
    'Person.CountryRegion',
    'Person.EmailAddress',
    'Person.Password',
    'Person.Person',
    'Person.PersonPhone',
    'Person.PhoneNumberType',
    'Person.StateProvince',
    'Production.BillOfMaterials',
    'Production.Culture',
    'Production.Document',
    'Production.Illustration',
    'Production.Location',
    'Production.Product',
    'Production.ProductCategory',
    'Production.ProductCostHistory',
    'Production.ProductDescription',
    'Production.ProductDocument',
    'Production.ProductInventory',
    'Production.ProductListPriceHistory',
    'Production.ProductModel',
    'Production.ProductModelIllustration',
    'Production.ProductModelProductDescriptionCulture',
    'Production.ProductPhoto',
    'Production.ProductProductPhoto',
    'Production.ProductReview',
    'Production.ProductSubcategory',
    'Production.ScrapReason',
    'Production.TransactionHistory',
    'Production.TransactionHistoryArchive',
    'Production.UnitMeasure',
    'Production.WorkOrder',
    'Production.WorkOrderRouting',
    'Purchasing.ProductVendor',
    'Purchasing.PurchaseOrderDetail',
    'Purchasing.PurchaseOrderHeader',
    'Purchasing.ShipMethod',
    'Purchasing.Vendor',
    'Sales.CountryRegionCurrency',
    'Sales.CreditCard',
    'Sales.Currency',
    'Sales.CurrencyRate',
    'Sales.Customer',
    'Sales.PersonCreditCard',
    'Sales.SalesOrderDetail',
    'Sales.SalesOrderHeader',
    'Sales.SalesOrderHeaderSalesReason',
    'Sales.SalesPerson',
    'Sales.SalesPersonQuotaHistory',
    'Sales.SalesReason',
    'Sales.SalesTaxRate',
    'Sales.SalesTerritory',
    'Sales.SalesTerritoryHistory',
    'Sales.ShoppingCartItem',
    'Sales.SpecialOffer',
    'Sales.SpecialOfferProduct',
    'Sales.Store'
]

aenc_tables = [
    'bonus',
    'customer',
    'department',
    'employee',
    'product',
    'region',
    'sales_order',
    'sales_order_item',
    'state'
]

transfer_tables_to_sqlite(northwind_tables, northwind_ssms_conn, northwind_sqlite_conn)
transfer_tables_to_sqlite(adventureworks_tables, adventureworks_ssms_conn, adventureworks_sqlite_conn)
transfer_tables_to_sqlite(aenc_tables, aenc_access_conn, aenc_sqlite_conn)

2024-05-06 15:41:01.984 | INFO     | __main__:transfer_tables_to_sqlite:5 - Transferring tables to SQLite database northwind.sqlite...
2024-05-06 15:41:02.112 | SUCCESS  | __main__:transfer_tables_to_sqlite:23 - Transferred tables to SQLite database northwind.sqlite!
2024-05-06 15:41:02.113 | INFO     | __main__:transfer_tables_to_sqlite:5 - Transferring tables to SQLite database adventureworks.sqlite...
2024-05-06 15:41:09.448 | SUCCESS  | __main__:transfer_tables_to_sqlite:23 - Transferred tables to SQLite database adventureworks.sqlite!
2024-05-06 15:41:09.448 | INFO     | __main__:transfer_tables_to_sqlite:5 - Transferring tables to SQLite database aenc.sqlite...
2024-05-06 15:41:09.449 | ERROR    | __main__:transfer_tables_to_sqlite:16 - Error transferring table bonus to SQLite database aenc.sqlite. Error: 'NoneType' object has no attribute 'cursor'
2024-05-06 15:41:09.449 | WARNING  | __main__:transfer_tables_to_sqlite:21 - Tables that failed to transfer: ['bonus']. Rest of the t

In [22]:
import pyodbc
import pandas as pd

# Function to transfer tables from a source database to a SQL Server database
def transfer_tables_to_sql(tables, source_conn, sql_conn_str):
    sql_conn = pyodbc.connect(sql_conn_str)
    cursor = sql_conn.cursor()

    for table in tables:
        try:
            sql_query = f'SELECT * FROM {table};'
            data = pd.read_sql(sql_query, source_conn)
            sql_table_name = table.replace('[', '').replace(']', '')  # Remove square brackets for tables with a space
            data.to_sql(sql_table_name, sql_conn, index=False, if_exists='replace')
        except Exception as e:
            print(f'Error transferring table {table} to SQL Server database. Error: {e}')
            break

    cursor.close()
    sql_conn.close()

# Connection string for SQL Server database
sql_conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=server_name;DATABASE=db_name;UID=user;PWD=password'

# Transfer tables
transfer_tables_to_sql(northwind_tables, northwind_ssms_conn, sql_conn_str)
transfer_tables_to_sql(adventureworks_tables, adventureworks_ssms_conn, sql_conn_str)
transfer_tables_to_sql(aenc_tables, aenc_access_conn, sql_conn_str)

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

In [16]:
northwind_sqlite_conn.close()
adventureworks_sqlite_conn.close()
northwind_ssms_conn.close()
adventureworks_ssms_conn.close()
#aenc_access_conn.close()
aenc_sqlite_conn.close()